In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
import h5py


gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(f"GPU Detected: {gpu.device_type} - {gpu.name}")
else:
    print("No GPU detected")

No GPU detected


# All data

In [3]:
# Open the HDF5 file
hdf5_file = 'processed_images_all.hdf5'
with h5py.File(hdf5_file, 'r') as file:
    # Read the test labels dataset into a numpy array
    test_labels = file['test_images_labels'][:]

# Create a DataFrame from the labels
test_labels = pd.DataFrame(test_labels, columns=['label'])

test_labels.shape

(3574, 1)

## Inspect images file content

In [4]:
# Function to inspect the contents of an HDF5 file
def inspect_hdf5_file(filepath):
    # Open the HDF5 file
    with h5py.File(filepath, 'r') as file:
        print("Datasets contained within the file:")
        for dataset_name in file:
            # Access the dataset
            dataset = file[dataset_name]
            # Print details about the dataset
            print(f"Dataset Info: {dataset_name}, {dataset.shape}, {dataset.dtype}")

# Call the function to inspect the HDF5 file
inspect_hdf5_file(hdf5_file)

Datasets contained within the file:
Dataset Info: test_images, (3574, 224, 224, 3), float64
Dataset Info: test_images_labels, (3574,), int32
Dataset Info: train_images, (28561, 224, 224, 3), float64
Dataset Info: train_images_labels, (28561,), int32
Dataset Info: valid_images, (3579, 224, 224, 3), float64
Dataset Info: valid_images_labels, (3579,), int32


## Image generator function

In [5]:
# Define a generator function to load images from the HDF5 file
def hdf5_image_generator(filepath, dataset_name, batch_size):
    while True:  # Loop indefinitely
        with h5py.File(filepath, 'r') as file:
            images = file[dataset_name]
            labels = file[dataset_name + '_labels']  # Adjust if your labels are stored differently
            num_samples = images.shape[0]

            for start in range(0, num_samples, batch_size):
                end = min(start + batch_size, num_samples)
                yield images[start:end], labels[start:end]

batch_size = 32

# Create generators for training and validation
train_generator = hdf5_image_generator(hdf5_file, 'train_images', batch_size)
valid_generator = hdf5_image_generator(hdf5_file, 'valid_images', batch_size)
test_generator = hdf5_image_generator(hdf5_file, 'test_images', batch_size)

# Get the lengths of the datasets
def get_dataset_lengths(filepath):
    with h5py.File(filepath, 'r') as file:
        lengths = {}
        for dataset_name in file:
            dataset = file[dataset_name]
            lengths[dataset_name] = len(dataset)
    return lengths

dataset_lengths = get_dataset_lengths(hdf5_file)
train_steps_per_epoch = dataset_lengths['train_images'] // batch_size
valid_steps = dataset_lengths['valid_images'] // batch_size
test_steps = dataset_lengths['test_images'] // batch_size

## Early stopping callback

In [6]:
# Implement EarlyStopping callback function
early_stopping = EarlyStopping(
    monitor='val_loss',   # Monitor the validation loss
    patience=10,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,            # Log when training stops
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored metric
)

## Model

In [7]:
# Load ResNet50 model, similar adjustments as VGG16
resnet50_base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = Flatten()(resnet50_base_model.output)

x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

output = Dense(10, activation='softmax')(x)

resnet50_model = Model(inputs=resnet50_base_model.input, outputs=output)
resnet50_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Fit model

In [ ]:
# Fit the model
resnet50_model_history = resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=50,
    batch_size=32,
    validation_data=valid_generator,
    validation_steps=valid_steps,
    #callbacks=[early_stopping]  # Include the early stopping callback
)

### Save model

In [ ]:
resnet50_model.save('models/resnet50_model_all.keras')

### Load model

In [ ]:
# Load the model from the .h5 file
#resnet50_model = load_model('models/resnet50_model_all.keras')

d:\Julio\Uc3m\TFG\venv\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


### Evaluate

In [ ]:
# Perform evaluation
test_loss, test_acc = resnet50_model.evaluate(test_generator, steps=test_steps)
print("ResNet50 Model Test Accuracy:", test_acc)

### Plot

In [ ]:
# Plot the history
# Assuming 'history' is the output from your model.fit()
plt.figure(figsize=(14, 5))

# Plot training & validation loss values
plt.subplot(1, 2, 1)
plt.plot(resnet50_model_history.history['loss'], label='Train Loss')
plt.plot(resnet50_model_history.history['val_loss'], label='Validation Loss')
plt.title('ResNet50 Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# Plot training & validation mean squared error
plt.subplot(1, 2, 2)
plt.plot(resnet50_model_history.history['accuracy'], label='Train accuracy')
plt.plot(resnet50_model_history.history['val_accuracy'], label='Validation accuracy')
plt.title('ResNet50 Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
predictions = resnet50_model.predict(test_generator, steps=test_steps)
predicted_labels = np.argmax(predictions, axis=1)
test_labels, predicted_labels

In [ ]:
accuracy = accuracy_score(test_labels, predicted_labels)
print("ResNet50 Model Accuracy:", accuracy)

### Confusion Matrix

In [ ]:
# Get the confusion matrix
conf_matrix = confusion_matrix(test_labels, predicted_labels)
print("ResNet50 Model Confusion Matrix:\n", conf_matrix)

# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
ax = sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=emotion_labels.values(), yticklabels=emotion_labels.values(), cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('ResNet50 Model Confusion Matrix')

# Move the X-axis to the top of the plot
ax.xaxis.tick_top()  # X-axis on top
ax.xaxis.set_label_position('top')  # Move the x-axis label to the top

plt.show()